In [7]:
def nonify(func):
    def wrapper(*args, **kwargs):
        if func(*args, **kwargs):
            return func(*args, **kwargs)
        else:
            return 
    return wrapper

In [20]:
@nonify
def aNb(a, n, b):
    return a*n+b

print(aNb(1,2,3), aNb("QWE",0,""))

5 None


In [74]:
import functools


def fix(number):
    def wrap(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            new_args, new_kwargs = [], {}
            for arg in args:
                new_args.append(round(arg, number) if isinstance(arg, float) else arg)
            for key, value in kwargs.items():
                new_kwargs[key] = round(value, number) if isinstance(value, float) else value
            return round(func(*new_args, **new_kwargs), number)
        return wrapper
    return wrap


In [70]:
@fix(4)
def aver(*args, sign=1):
    return sum(args)*sign

print(aver(2.45675901, 3.22656321, 3.432654345, 4.075463224, sign=-1))

-13.1916


In [8]:
def biased(init):
    bias = yield init
    while bias:
        init += bias*2+1
        bias = yield init
g = biased(10)
next(g) # или, что то же самое, g.send(None)

g.send(5)

g.send(5)

g.send(-1)

g.send(100500)

# g.send(0)

201032

In [24]:
def turtle(coord, direction):
    di = direction
    x, y = coord
    cmd = yield coord
    while cmd:
        if cmd == 'l':
            di = (di + 1) % 4
        elif cmd == 'r':
            di = (di + 3) % 4
        elif cmd == 'f':
            if di == 0:
                x += 1
            elif di == 1:
                y += 1
            elif di == 2:
                x -= 1
            elif di == 3:
                y -= 1
        cmd = yield x, y

# g = turtle((0, 0), 0)
# g.send(None)
# g.send('f')


In [25]:
robo = turtle((0,0),0)
start = next(robo)
for c in "flfrffrffr":
    print(*robo.send(c))

1 0
1 0
1 1
1 1
2 1
3 1
3 1
3 0
3 -1
3 -1


Написать, держитесь крепче, генератор-декоратор statcounter(), который конструирует объекты (назовём один из них stat) со следующим поведением. Первый вызов next(stat) (он же stat.send(None)) возвращает словарь, в котором stat будет хранить информацию вида функция: количество вызовов, где функция — это исходный (не обёрнутый) объект-функция (да, так тоже можно!). Все последующие вызовы stat.send(function) оборачивают вызов произвольной функции function увеличением на 1 соответствующего элемента словаря. Глобальными именами пользоваться нельзя. В примере видны уникальные id объектов, в тестах их не будет (я воспользуюсь function.__name__ или просто не буду их учитывать).

In [148]:
def statcounter(func):
    dic = {}
    yield dic
    while func:
        print('while', func)
        def decorator(func):
            print('decorator')
            def wrapper():
                print('wrapper')
                if func in list(dic.keys()):
                    dic[func] += 1
                else:
                    dic[func] = 1
            return wrapper
        return decorator
        print('dic', dic)
        func = yield dic

In [149]:
stat = statcounter()

TypeError: statcounter() missing 1 required positional argument: 'func'

In [150]:
stat.send(None)

StopIteration: 

In [151]:
@statcounter
def f():
    print('help')
    
a = f

def ff():
    print('HELP')

In [152]:
a.send(None)


{}

In [154]:
a.send(ff)

while <function ff at 0x000001F54A7537B8>
dic {}


{}

In [70]:
stat.send(function)

NameError: name 'function' is not defined